# Calculate diversity of generations
From paper, page 9:

The diversity is measured using the average overlap of the longest sub-sequence (LCS) among sampled answers:

\begin{equation}
1 - \frac{1}{\binom{M}{2}} \sum_{s \neq s' \in C} \text{ROUGE-L}(s, s') 
\end{equation}

$\binom{M}{2}$ is the number of possible pairs of $s \neq s'$ (order doesn't matter)


1) Calculate diversity of 5 groups 
2) For multinomial sampling and multinomial beam sampling

In [1]:
import glob
import pickle
from itertools import combinations
from rouge_score import rouge_scorer
import yaml
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

In [2]:
def diversity(string_list):
    rouge_l_sum = 0
    n_pairs = 0

    for s, s_prime in combinations(string_list, 2):
        rouge_l_sum += scorer.score(s, s_prime)["rougeL"].fmeasure
        n_pairs += 1

    return 1 - rouge_l_sum / n_pairs

In [3]:
def load_pickle_files(folder):
    data_groups = []
    pickle_files = glob.glob(f"{folder}/group*.pkl")
    for pickle_file in pickle_files:
        with open(pickle_file, "rb") as f:
            data_groups.append(pickle.load(f))

    return data_groups

In [4]:
# https://thepythoncode.com/article/calculate-rouge-score-in-python#rouge-l
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

save_path = config["path_to_saved_generations"]
data_groups = load_pickle_files(save_path)

keys_data = [f"temperature_{temp}" for temp in config["temperatures"]] + [f"beam_{beam}" for beam in config["n_beams"]]
diversity_scores = {k: [] for k in keys_data}

In [5]:
for data in data_groups:
    group_diversity_scores = {k: [] for k in keys_data}
    for question_idx in data.keys():
        for k in keys_data:
            answers = data[question_idx][k]["answers"]
            group_diversity_scores[k].append(diversity(answers))

    # Add mean diversity of group to diversity_scores
    diversity_scores = {k: diversity_scores[k] + [np.mean(group_diversity_scores[k])] for k in keys_data}

In [6]:
# Convert to pandas df
diversity = pd.DataFrame(diversity_scores,
                         index=pd.Index([f"group_{i}" for i in range(len(data_groups))]))
diversity.index.name = "group"
print("Diversity scores of sampled answers")
diversity

Diversity scores of sampled answers


,temperature_0.25,temperature_0.5,temperature_1,temperature_1.5,beam_20
group,,,,,
group_0,0.369426,0.637823,0.886433,0.956363,0.599882
group_1,0.347955,0.618786,0.883092,0.958035,0.611705
group_2,0.353941,0.625156,0.885565,0.959431,0.609484
group_3,0.356296,0.622342,0.888287,0.961473,0.617475
group_4,0.350981,0.605766,0.877252,0.955160,0.611195
